In [4]:
import pandas as pd
import numpy as np
import lr
def fillna_mean(df):
    for column in list(df.columns[df.isnull().sum() > 0]):
        mean_val = df[column].mean()
        df[column].fillna(mean_val, inplace=True)
    return df

# 读取数据


## 选出对应患者24h内的数据

In [24]:
# cxr = pd.read_csv("mimic-iv-cxr-2/train_set.csv", index_col=0)
# admission = pd.read_csv("mimic-iv-2/hosp/admissions.csv", header=0)
# admission = pd.merge(admission, cxr[["subject_id", "hadm_id", "label"]])
# admission["first_day"] = [i[0:11].replace("-","") for i in admission["admittime"]]
# lab = pd.read_csv("mimic-iv-2/hosp/labevents.csv")
# lab_d = pd.merge(admission[["subject_id", "first_day"]], lab)
# lab_d["time"] = [i[0:11].replace("-","") for i in lab_d["charttime"]]
# lab_d["hadm_id"] = lab_d["hadm_id"].fillna(0)
# lab_d["hadm_id"] =[int(i) for i in lab_d["hadm_id"]]
# lab_d["time_re"] = lab_d["first_day"].astype(int) - lab_d["time"].astype(int) 
# lab_d = lab_d.loc[(lab_d["time_re"]>=-1) & (lab_d["time_re"]<=1)] # 入院时间和检测时间相差1天内的
# lab_d.to_csv("predict_model/labevents.csv")

In [5]:
# 选出患者足够量的数据
lab_d = pd.read_csv("predict_model/labevents.csv", index_col=0)
# a = lab_d[['subject_id','hadm_id',"time_re"]].groupby(["subject_id","hadm_id"]).agg("max").reset_index()
lab_d = lab_d.loc[(lab_d["time_re"]==0)] # 入院时间和检测时间相差1天内的
lab_dd = lab_d[['subject_id','hadm_id','itemid', 'time', 'valuenum']]
lab_dd = lab_dd.loc[~lab_dd["valuenum"].isnull()].drop_duplicates()
effect_item = lab_dd["itemid"].value_counts()[lab_dd["itemid"].value_counts() > 1000]
lab_dd = lab_dd.loc[lab_dd["itemid"].isin(effect_item.index)]
lab_dd = lab_dd.groupby(["subject_id","hadm_id","time","itemid"]).agg('mean')
lab_dd = lab_dd.reset_index()
lab_dd["index"] = lab_dd["subject_id"].astype(str) + "_" + lab_dd["time"].astype(str)
lab_dd = lab_dd.pivot(index=["subject_id","hadm_id", "time"],columns='itemid',values="valuenum").reset_index()
d_lab = pd.read_csv("mimic-iv-2/hosp/d_labitems.csv")
new_idx  = list(lab_dd.columns[0:3])
new_idx.extend([d_lab.loc[d_lab["itemid"]==i,"label"].iloc[0] for i in lab_dd.columns[3:]])
lab_dd.columns = new_idx
# 添加CXR数据
cxr = pd.read_csv("mimic-iv-cxr-2/prediction_10fold.csv")
cxr.columns =['study_id', 'prediction', 'rate', 'cut'] 
cxr_train = pd.read_csv("mimic-iv-cxr-2/train_set.csv", index_col=0)
cxr_prediction = pd.merge(cxr_train,cxr)
full_df = pd.merge(lab_dd, cxr_prediction[["subject_id","label"]])
full_df.columns = [i.replace(" ", "_").replace("(","_").replace(")","_").replace(",","_")  for i in full_df.columns]

## LR模型

In [6]:
a = [
    'Base_Excess', 'Calculated_Total_CO2',
    'Lactate', 'pCO2', 'pH', 'pO2', 'Anion_Gap', 'Bicarbonate',
    'Calcium__Total', 'Chloride', 'Creatinine', 'Glucose', 'Magnesium',
    'Phosphate', 'Potassium', 'Sodium', 'Urea_Nitrogen', 'Basophils',
    'Eosinophils', 'Hematocrit', 'Hemoglobin', 'INR_PT_', 'Lymphocytes',
    'MCH', 'MCHC', 'MCV', 'Monocytes', 'Neutrophils', 'Platelet_Count',
    'PT', 'PTT', 'RDW', 'Red_Blood_Cells', 'White_Blood_Cells', 'pH',
    'Specific_Gravity',]

### stats.model的lr用于feature selection


In [9]:
file = full_df
file.loc[file["label"]=="yes", "label"] = int(1)
file.loc[file["label"]=="no", "label"] = int(0)
file["label"]=np.array(file["label"],dtype=np.float)
lr.sm_lr(file, a)

/tmp/ipykernel_4141901/2086305229.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  file["label"]=np.array(file["label"],dtype=np.float)


         Current function value: 0.123647
         Iterations: 35
         Function evaluations: 46
         Gradient evaluations: 46
1.459627186898162e+02
                           Logit Regression Results                           
Dep. Variable:                  label   No. Observations:                  291
Model:                          Logit   Df Residuals:                      254
Method:                           MLE   Df Model:                           36
Date:                Mon, 26 Dec 2022   Pseudo R-squ.:                  0.7004
Time:                        10:18:37   Log-Likelihood:                -35.981
converged:                      False   LL-Null:                       -120.11
Covariance Type:            nonrobust   LLR p-value:                 5.409e-19
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.0621   

/home/data/t070224/miniconda3/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))
/home/data/t070224/miniconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


(<statsmodels.discrete.discrete_model.BinaryResultsWrapper at 0x7ffb5a4d89d0>,
 <class 'statsmodels.iolib.table.SimpleTable'>)

### 获取AUC

In [ ]:
label_l, prepro_l = lr.sk_lr(file, a, [])
lr.auc(label_l, prepro_l)